#Project 1 - Report 1 code

In [2]:
#Please have the 'playlists4000.txt' file in the same directory as this notebook.

fd = open("playlists4000.txt",mode='r',encoding='utf8',newline='\n')

raw = fd.read().splitlines()

hashtable = dict()

for line in raw:
    first, second = line.split(sep="::")
    hashtable.setdefault(first, []).append(second)

hashtable

{'0': ['0'],
 '1': ['1', '2', '3', '4'],
 '2': ['5', '6', '7'],
 '3': ['8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19'],
 '4': ['20', '21', '22', '4'],
 '5': ['23'],
 '6': ['24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44',
  '45',
  '46',
  '47',
  '48',
  '49'],
 '7': ['50',
  '51',
  '28',
  '52',
  '53',
  '54',
  '55',
  '56',
  '57',
  '58',
  '59',
  '60',
  '61',
  '62',
  '63',
  '64',
  '65',
  '66',
  '67',
  '68',
  '69',
  '70',
  '71',
  '41',
  '72',
  '73',
  '74',
  '75',
  '76',
  '77',
  '78',
  '47',
  '79',
  '80',
  '81'],
 '8': ['82', '83'],
 '9': ['84',
  '85',
  '86',
  '87',
  '88',
  '89',
  '90',
  '91',
  '92',
  '93',
  '94',
  '95',
  '96',
  '97',
  '98',
  '99',
  '100',
  '101',
  '102',
  '103',
  '104',
  '105',
  '106',
  '107',
  '6',
  '108',
  '109',
  '110',
  '111',
  '112',
  '113',
  '114',
  '115',
  '116',


In [3]:
#Number of items
items = 0
for x in hashtable.values():
    items += len(x)
items

55994

In [4]:
#Number of transactions
len(hashtable.keys())

4000

In [8]:
#Longest transaction
max = 0
for x in hashtable.values():
    if (len(x) > max):
        max = len(x)

max

110

In [9]:
#Shortest transaction
min = max
for x in hashtable.values():
    if(len(x) < min):
        min = len(x)
min

1

In [10]:
one_itemsets = dict()

for x in hashtable.values():
    if (len(x) == 1):
        one_itemsets
        

dict_values([['0'], ['1', '2', '3', '4'], ['5', '6', '7'], ['8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19'], ['20', '21', '22', '4'], ['23'], ['24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49'], ['50', '51', '28', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '41', '72', '73', '74', '75', '76', '77', '78', '47', '79', '80', '81'], ['82', '83'], ['84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '6', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118'], ['28', '119', '57', '60', '120', '121', '122', '123', '124', '125', '126', '80'], ['24', '127', '128', '129', '130', '131'], ['132', '133', '134', '135', '136', '137', '138'], ['139', '140', '141', '127', '142', '143